In [1]:
# We use pandas for data manipulation, and os to handle file paths
import pandas as pd
import os


In [4]:
df = pd.read_csv("../data/raw/diabetic_data.csv")
df.shape


(101766, 50)

In [5]:
df.head()


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
df.replace("?", pd.NA, inplace=True)
df.isna().sum().sort_values(ascending=False).head(10)


weight               98569
max_glu_serum        96420
A1Cresult            84748
medical_specialty    49949
payer_code           40256
race                  2273
diag_3                1423
diag_2                 358
diag_1                  21
encounter_id             0
dtype: int64

In [7]:
df.drop(columns=[
    "encounter_id",              # Unique for every hospital visit
    "patient_nbr",               # Unique for each patient (not useful for prediction)
    "examide",                   # Always "No" – zero variance
    "citoglipton",               # Always "No" – zero variance
    "payer_code",                # Mostly missing
    "medical_specialty",        # > 50% missing
    "weight"                    # > 90% missing
], inplace=True)


In [8]:
df.shape

(101766, 43)

In [9]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [10]:
df.isna().sum().sort_values(ascending=False)

max_glu_serum               96420
A1Cresult                   84748
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
acetohexamide                   0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose                        0
miglitol                        0
troglitazone                    0
tolazamide                      0
insulin                         0
glyburide-metformin             0
glipizide-metformin             0
glimepiride-pioglitazone        0
metformin-rosiglitazone         0
metformin-pioglitazone          0
change                          0
diabetesMed                     0
glipizide                       0
nateglinide                     0
glimepiride                     0
chlorpropamide                  0
age                             0
admission_type_id               0
discharge_disp

In [12]:
df.drop(columns=[
                                 # Missing
    "max_glu_serum",       # ~95% missing
    "A1Cresult",           # ~83% missing
    "diag_3"               # Less important + missing
], inplace=True)


In [13]:
# Turn '[60-70)' into 65 (midpoint)
def convert_age(age_str):
    return int(age_str.strip("[]").split("-")[0]) + 5

df["age"] = df["age"].apply(convert_age)
df["age"].unique()


array([ 5, 15, 25, 35, 45, 55, 65, 75, 85, 95])

In [14]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,5,6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,15,1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,25,1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,35,1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,45,1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [15]:
df.isna().sum().sort_values(ascending=False)

race                        2273
diag_2                       358
diag_1                        21
troglitazone                   0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitazone                  0
acarbose                       0
miglitol                       0
tolazamide                     0
acetohexamide                  0
insulin                        0
glyburide-metformin            0
glipizide-metformin            0
glimepiride-pioglitazone       0
metformin-rosiglitazone        0
metformin-pioglitazone         0
change                         0
diabetesMed                    0
glipizide                      0
glimepiride                    0
gender                         0
chlorpropamide                 0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_proced

In [18]:
df.isna().sum(axis=1).value_counts().sort_index()


0    99139
1     2602
2       25
Name: count, dtype: int64

In [19]:
threshold = int(df.shape[1] * 0.8)  # require at least 80% non-missing
df.dropna(thresh=threshold, inplace=True)
df.shape


(101766, 40)

In [20]:
# '<30' means readmitted within 30 days — we treat it as 1
df["readmitted"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)
df["readmitted"].value_counts()


readmitted
0    90409
1    11357
Name: count, dtype: int64

In [21]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,5,6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,15,1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,25,1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,35,1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,45,1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,0


In [24]:
os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/diabetes_cleaned.csv", index=False)
print("✅ Cleaned dataset saved to data/processed/diabetes_cleaned.csv")


✅ Cleaned dataset saved to data/processed/diabetes_cleaned.csv


In [25]:

# ✅ Save final cleaned version
df.to_csv("data/processed/diabetes_with_numeric_age.csv", index=False)
print("✅ Saved diabetes_with_numeric_age.csv")


✅ Saved diabetes_with_numeric_age.csv
